In [1]:
import numpy as np
import pandas as pd
import quantecon as qe
from ast import literal_eval
from sklearn.ensemble import RandomForestRegressor

In [2]:
def creating_samples(samples_number, element):
    gen=0
    uniform_number = len(element)
    uniform_samples = (-qe.simplex_grid(len(element), 1)+1)/(len(element)-1)
    if samples_number > uniform_number:
        random_samples = np.random.rand((samples_number-uniform_number), len(element))
        for i in range(samples_number-uniform_number):
            random_samples[i] = np.around(random_samples[i]/sum(random_samples[i]), decimals = 3)
        sample = np.vstack((uniform_samples, random_samples))
    else:
        sample = uniform_samples[:samples_number]
    samples = []
    for i in range(samples_number):
        samples.append(list(sample[i]))
    samples = np.array(samples)
    generation = []
    for i in range (samples_number):
            generation.append(gen)

    ID = np.arange(1, (samples_number+1))

    data = {'ID' : ID, 'Elements': [element], 'Generation': generation}
    df = pd.DataFrame(data=data, index = np.arange(samples_number))
    df = pd.concat([df, pd.DataFrame(([[i] for i in samples]), columns = ['Position'])], axis = 1)
    df.to_csv("Result/Initial Population.txt", sep='\t', index=False, mode='w')
    return df

In [3]:
#Loading data and fitting
df_old = pd.read_csv('Data/AuIrOsPdPtReRhRu_0.60_compositions_and_targets.csv')
X_columns_old = ['Pt','Pd','Au','Ru','Rh','Ir','Re','Os']
x_old = df_old[X_columns_old].to_numpy()
y_old = df_old['current_over_capacitance'].to_numpy()

#Rndom Forest Regression
reg = RandomForestRegressor(n_estimators = 1024,
                           bootstrap = True,
                           max_depth = None,
                           max_features = 'auto',
                           min_samples_leaf = 1,
                           min_samples_split = 2,
                           oob_score = True)
reg = reg.fit(x_old, y_old)

In [4]:
#Determine the best position
grid = qe.simplex_grid(3, 100)/100
zeros = np.zeros(len(grid)).reshape(-1,1)

grid = np.concatenate((grid, zeros), axis=1)
grid = np.concatenate((grid, zeros), axis=1)
grid = np.concatenate((grid, zeros), axis=1)
grid = np.concatenate((grid, zeros), axis=1)
grid = np.concatenate((grid, zeros), axis=1)
grid = pd.DataFrame(grid,columns=[X_columns_old])
grid['Activity'] = reg.predict(grid.to_numpy())
best_grid = grid.iloc[np.argmin(grid['Activity'])][0:3].to_numpy()

In [38]:
#Making Class
class pso:
    def __init__ (self, data, step, target = np.array([0.333, 0.333, 0.333])):
        self.datalog = data
        #Reading "Elements" columns from string to list
        self.datalog['Elements'] = self.datalog["Elements"].apply(lambda x: literal_eval(x))
        
        #Select latest generation
        self.generation = self.datalog['Generation'].max()
        
        #Creating np.array of "Position" column and dropping the string type "Position" column
        self.position = []
        for i in range(self.datalog['ID'].max()):
            self.position.append(list(np.fromstring(self.datalog['Position'][i][1:-1], dtype=float, sep=' ')))
        self.position = np.array(self.position)
        self.datalog = self.datalog.drop(columns=['Position'])
        self.datalog = pd.concat([self.datalog, pd.DataFrame(([[i] for i in self.position]), columns = ['Position'])], axis = 1)
        
        #Creating "Velocity" column for the "0" generation
        if self.generation == 0:
            self.velocity = np.around((-(self.position - target)/step), decimals = 3)
            self.datalog = pd.concat([self.datalog, pd.DataFrame(([[i] for i in self.velocity]), columns = ['Velocity'])], axis = 1)

        
        #Creating blank "Activity" column
        self.datalog = pd.concat([self.datalog, pd.DataFrame(columns = ['Activity'], index = np.arange(self.datalog['ID'].max()))], axis = 1)      

        #Filling the "Activity" column with RFR
        self.f_activity(self.datalog)
            
        #Creating dataframe of the latest generation
        self.working_generation = self.datalog.loc[self.datalog['Generation']==self.generation]
    
    
            
    def move(self, factor_1, factor_2, factor_3):
        #Performing move function  
        self.generation += 1
        self.working_generation['Generation'] += 1
        for i in range(len(self.working_generation)):
            new_velocity = -(factor_1 * population.working_generation.at[i, 'Velocity'] + 
                        factor_2 * population.delta_gen_best((i+1), 'Activity') * population.delta_gen_best((i+1), 'Position') + 
                        factor_3 * population.delta_individual_best((i+1), 'Activity') * population.delta_individual_best((i+1), 'Position'))
            self.working_generation.at[i,'Velocity'] = new_velocity
        self.normalize_velocity()
            
        for i in range(len(self.working_generation)):    
            new_position = self.working_generation['Position'][i] + self.working_generation['Velocity'][i]
            self.working_generation.at[i,'Position'] = np.around(new_position, decimals = 3)
            
            
            
        """#Checking whether the new position cross the boundaries (UNFINISHED)
        if new_position.max() > 1 or new_position.min() < 0:
            new_position = self.working_generation['Position'][i] - self.working_generation['Velocity'][i]*size
        self.working_generation.at[i,'Position'] = np.around(new_position, decimals = 3)"""
        
        
        #Filling the "Activity" column with RFR
        self.f_activity(self.working_generation)
        
        #Concating the tables
        self.datalog = pd.concat([self.datalog, self.working_generation])
        self.datalog = self.datalog.reset_index(drop=True)
        self.store_datalog()
        return
    
    def f_activity(self, dataframe):
        global reg
        a = [0, 0, 0, 0, 0]
        for i in range(len(dataframe)):
            dataframe.at[i, 'Activity'] = float(reg.predict(np.reshape((np.hstack((dataframe.at[i, 'Position'], a))), (1, -1))))
        return 

    def store_datalog(self):
        self.datalog.to_csv("Result/Initial Population_gen "+str(self.generation)+".txt", sep='\t', index=False, mode='w')
        return
    
    def normalize_velocity(self):
        for i in range(len(self.working_generation)):
            self.working_generation.at[i, 'Velocity'] = self.working_generation.at[i, 'Velocity'] - (sum(self.working_generation.at[i, 'Velocity'])/len(self.working_generation['Elements'][0]))
            self.working_generation.at[i, 'Velocity'] = np.around(self.working_generation.at[i, 'Velocity'], decimals= 3)
        return
    
    def normalize_position(self):
        for i in range(len(self.working_generation)):
            self.working_generation.at[i, 'Position'] = self.working_generation.at[i, 'Position']/(sum(self.working_generation.at[i, 'Position']))
            self.working_generation.at[i, 'Position'] = np.around(self.working_generation.at[i, 'Position'], decimals= 3)
        return 
    
    
    def gen_best(self):
        return self.working_generation.loc[np.argmin(self.working_generation['Activity'])]
           
   
    def individual_best(self, ID):
        self.i_data = self.datalog[self.datalog['ID']==ID].reset_index(drop=True)
        return self.i_data.loc[np.argmin(self.i_data['Activity'])]
    
    def delta_gen_best(self, ID, parameter):
        return self.working_generation[self.working_generation['ID']==ID][parameter].reset_index(drop=True)[0] - self.gen_best()[parameter]
    
    def delta_individual_best(self, ID, parameter):
        return self.working_generation[self.working_generation['ID']==ID][parameter].reset_index(drop=True)[0] - self.individual_best(ID)[parameter]      
    

In [51]:
samples_number = 6
element = ['Pt', 'Pd', 'Au']
creating_samples(samples_number, element)

,ID,Elements,Generation,Position
0,1,"[Pt, Pd, Au]",0,"[0.5, 0.5, 0.0]"
1,2,"[Pt, Pd, Au]",0,"[0.5, 0.0, 0.5]"
2,3,"[Pt, Pd, Au]",0,"[0.0, 0.5, 0.5]"
3,4,"[Pt, Pd, Au]",0,"[0.391, 0.29, 0.32]"
4,5,"[Pt, Pd, Au]",0,"[0.305, 0.315, 0.379]"
5,6,"[Pt, Pd, Au]",0,"[0.484, 0.222, 0.295]"


In [52]:
step = 10
target = np.array([0.333, 0.333, 0.333])
population = pso(pd.read_csv('Result/Initial Population.txt', sep='\t'), step, target)

In [53]:
population.datalog

,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",0,"[0.5, 0.5, 0.0]","[-0.017, -0.017, 0.033]",-0.931174
1,2,"[Pt, Pd, Au]",0,"[0.5, 0.0, 0.5]","[-0.017, 0.033, -0.017]",-0.904198
2,3,"[Pt, Pd, Au]",0,"[0.0, 0.5, 0.5]","[0.033, -0.017, -0.017]",-0.264023
3,4,"[Pt, Pd, Au]",0,"[0.391, 0.29, 0.32]","[-0.006, 0.004, 0.001]",-1.027099
4,5,"[Pt, Pd, Au]",0,"[0.305, 0.315, 0.379]","[0.003, 0.002, -0.005]",-0.98878
5,6,"[Pt, Pd, Au]",0,"[0.484, 0.222, 0.295]","[-0.015, 0.011, 0.004]",-0.999214


In [55]:
factor_1 = 0.7
factor_2 = 1.5
factor_3 = 0.8
for i in range(20):
    population.move(factor_1, factor_2, factor_3)
    display(population.working_generation)
    print(population.gen_best()['Position'], population.gen_best()['Activity'])
    print(np.sqrt(sum((population.gen_best()['Position']-best_grid)**2)))

,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",2,"[0.484, 0.467, 0.049]","[-0.012, -0.014, 0.026]",-0.934912
1,2,"[Pt, Pd, Au]",2,"[0.48, 0.056, 0.464]","[-0.012, 0.026, -0.014]",-0.908364
2,3,"[Pt, Pd, Au]",2,"[0.127, 0.432, 0.442]","[-0.297, 0.161, 0.137]",-0.77587
3,4,"[Pt, Pd, Au]",2,"[0.392, 0.289, 0.32]","[-0.003, 0.002, 0.001]",-1.027099
4,5,"[Pt, Pd, Au]",2,"[0.31, 0.313, 0.376]","[0.002, 0.001, -0.003]",-0.995911
5,6,"[Pt, Pd, Au]",2,"[0.482, 0.224, 0.296]","[-0.009, 0.007, 0.003]",-1.000147


[0.392 0.289 0.32 ] -1.027099003906248
0.09759610647971569


,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",3,"[0.48, 0.452, 0.068]","[-0.004, -0.015, 0.019]",-0.937695
1,2,"[Pt, Pd, Au]",3,"[0.473, 0.079, 0.448]","[-0.007, 0.023, -0.016]",-0.908119
2,3,"[Pt, Pd, Au]",3,"[0.491, 0.235, 0.274]","[0.364, -0.197, -0.168]",-1.005892
3,4,"[Pt, Pd, Au]",3,"[0.394, 0.288, 0.319]","[0.002, -0.001, -0.001]",-1.027099
4,5,"[Pt, Pd, Au]",3,"[0.312, 0.311, 0.375]","[0.002, -0.002, -0.001]",-0.995911
5,6,"[Pt, Pd, Au]",3,"[0.485, 0.222, 0.295]","[0.003, -0.002, -0.001]",-0.999214


[0.394 0.288 0.319] -1.027099003906248
0.09550392662084634


,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",4,"[0.471, 0.44, 0.088]","[-0.009, -0.012, 0.02]",-0.944226
1,2,"[Pt, Pd, Au]",4,"[0.464, 0.1, 0.436]","[-0.009, 0.021, -0.012]",-0.920975
2,3,"[Pt, Pd, Au]",4,"[0.233, 0.375, 0.393]","[-0.258, 0.14, 0.119]",-0.958475
3,4,"[Pt, Pd, Au]",4,"[0.393, 0.289, 0.32]","[-0.001, 0.001, 0.001]",-1.027099
4,5,"[Pt, Pd, Au]",4,"[0.314, 0.311, 0.373]","[0.002, 0.0, -0.002]",-0.995174
5,6,"[Pt, Pd, Au]",4,"[0.479, 0.226, 0.297]","[-0.006, 0.004, 0.002]",-1.00278


...

[0.394 0.289 0.32 ] -1.027099003906248
0.09621330469326995


,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",20,"[0.422, 0.346, 0.227]","[-0.001, -0.002, 0.003]",-0.999524
1,2,"[Pt, Pd, Au]",20,"[0.432, 0.185, 0.383]","[-0.001, 0.003, -0.002]",-0.988996
2,3,"[Pt, Pd, Au]",20,"[0.367, 0.304, 0.333]","[-0.001, 0.001, 0.001]",-1.020017
3,4,"[Pt, Pd, Au]",20,"[0.394, 0.289, 0.32]","[-0.0, -0.0, -0.0]",-1.027099
4,5,"[Pt, Pd, Au]",20,"[0.34, 0.303, 0.354]","[0.0, 0.0, -0.001]",-1.004035
5,6,"[Pt, Pd, Au]",20,"[0.459, 0.24, 0.302]","[-0.001, 0.0, 0.0]",-1.011233


[0.394 0.289 0.32 ] -1.027099003906248
0.09621330469326995


,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",21,"[0.421, 0.345, 0.229]","[-0.001, -0.001, 0.002]",-1.000031
1,2,"[Pt, Pd, Au]",21,"[0.431, 0.188, 0.381]","[-0.001, 0.003, -0.002]",-0.989327
2,3,"[Pt, Pd, Au]",21,"[0.368, 0.303, 0.332]","[0.001, -0.001, -0.001]",-1.020645
3,4,"[Pt, Pd, Au]",21,"[0.394, 0.289, 0.32]","[-0.0, -0.0, -0.0]",-1.027099
4,5,"[Pt, Pd, Au]",21,"[0.342, 0.302, 0.353]","[0.002, -0.001, -0.001]",-1.004766
5,6,"[Pt, Pd, Au]",21,"[0.458, 0.241, 0.302]","[-0.001, 0.001, 0.0]",-1.011282


[0.394 0.289 0.32 ] -1.027099003906248
0.09621330469326995


,ID,Elements,Generation,Position,Velocity,Activity
0,1,"[Pt, Pd, Au]",8,"[0.467, 0.469, 0.063]","[-0.019, -0.02, 0.039]",-0.937853
1,2,"[Pt, Pd, Au]",8,"[0.459, 0.121, 0.418]","[-0.013, 0.01, 0.003]",-0.975796
2,3,"[Pt, Pd, Au]",8,"[0.333, 0.392, 0.275]","[-0.225, 0.047, 0.178]",-0.991545
3,4,"[Pt, Pd, Au]",8,"[0.287, 0.492, 0.221]","[-0.005, -0.008, 0.014]",-0.953691
4,5,"[Pt, Pd, Au]",8,"[0.369, 0.374, 0.257]","[-0.004, -0.004, 0.008]",-1.011998
5,6,"[Pt, Pd, Au]",8,"[0.281, 0.342, 0.379]","[0.008, 0.001, -0.008]",-0.975415


0.12404031602668548
